# Re-identification and De-identification

In [290]:
import pandas as pd
import numpy as np

In [51]:
"""
Useful display function for dataframe
"""
def display_df(df, nrows=10, ncols=None):
    with pd.option_context('display.max_rows', nrows, 'display.max_columns', ncols):
        display (df)

## Import data

In [2]:
#whole unaltered dataset
df_raw = pd.read_csv("../mid_sample_set.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Drop Unnecessary Fields and Clean NaNs

In [65]:
"""
Reads configuration file, a list of strings seperated by new lines, and returns a list
"""
def read_config(file):
    with open(file) as f:
        config_list = [(l) for l in f.read().split()]
    f.close()
    return config_list

In [66]:
qis = read_config('config.txt')

In [67]:
qis

['cc_by_ip',
 'countryLabel',
 'continent',
 'city',
 'region',
 'subdivision',
 'postalCode',
 'LoE',
 'YoB',
 'gender',
 'nforum_posts',
 'nforum_votes',
 'nforum_endorsed',
 'nforum_threads',
 'nforum_comments',
 'nforum_pinned',
 'nforum_events']

We only need to keep the `user_id` as a key, the quasi-identifiers, the `completed` field to find the completion rate, and `explored` to find the exploration rate. Everything else can be dropped. Then we can clean the dataset.

In [359]:
df_clean = df_raw[['user_id'] + qis + ['completed', 'explored']]

Many of the fields contain NaNs when they actually should contain 0. We will replace those values.

In [88]:
"""
Takes list of fields with NaNs and fills NaN values with fill_val. Does this inplace.
"""
def replace_NaNs(df, labels, fill_val):
    for label in labels:
        df[label].fillna(fill_val, inplace=True)
"""
Gets ratio of NaNs for each column
"""
def stats_NaN(df):
    df_stats = pd.DataFrame(index=[df.columns], columns=["NaN Ratio"])
    for col in df.columns:
        df_stats["NaN Ratio"][col] = df[col].isna().sum()/len(df) #NaN ratio
    return df_stats.sort_values(by=['NaN Ratio'])

In [89]:
stats_NaN(df_clean)

,NaN Ratio
user_id,0
completed,0
continent,0.110371
countryLabel,0.111971
cc_by_ip,0.112171
gender,0.131326
LoE,0.139956
YoB,0.150226
nforum_events,0.184851
city,0.225491


In [360]:
NaN_to_0_fields = ['YoB', 'postalCode', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 
                   'nforum_threads', 'nforum_comments', 'nforum_pinned', 'nforum_events',
                  'explored']
replace_NaNs(df_clean, NaN_to_0_fields, 0)

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [361]:
#cast to numeric type
df_clean['explored'] = pd.to_numeric(df_clean['explored'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Add Useful Statistical Fields

In [362]:
df_clean.sort_values('user_id', inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In k-anonymizing a dataset, records will be compressed together. Therefire, we must preserve valuable statistics like the completion rate, which means we must create new columns `nStarted` and `nCompleted` which keeps track of the amount of classes started and completed respectively by a given user.

In [363]:
df_clean = df_clean.join(pd.DataFrame(df_clean.groupby('user_id').size(), 
                                      columns=['nStarted']),
                         on='user_id')

In [364]:
df_clean = df_clean.join(pd.DataFrame(df_clean[df_clean['completed']==True].groupby(['user_id']).size(), 
                                      columns=['nCompleted']),
                   on='user_id')

In [365]:
df_clean = df_clean.join(pd.DataFrame(df_clean[df_clean['explored']==True].groupby(['user_id']).size(), 
                                      columns=['nExplored']),
                   on='user_id')

In [366]:
#Fix NaNs in nCompleted row
replace_NaNs(df_clean, ['nCompleted','nExplored'], 0)

We can drop `completed` and `explored` as it is no longer necessary in analysis.

In [367]:
df_clean.drop(columns=['completed','explored'], inplace=True)

## Getting Completion Rate

We will write a generalizable function that finds the completion rate of a dataset. It will use the `nStarted` and `nCompleted` columns to tabulate this. It will be general enough to use on the clean dataset without double counting and also able to handle the k-anonymized datasets where we have already handled duplicate values.

In [375]:
"""
Returns completion rate and exploration rate of a dataframe. If user_id is present,
function counts per unique_id to avoid double counting. Otherwise assumes that
duplicates have been handled if user_id is dropped. Returns list. First element is 
completion rate, second element is exploration rate.
"""
def getStats(df):
    if 'user_id' in df.columns:
        df = df[['user_id', 'nStarted', 'nExplored', 'nCompleted']]
        df.drop_duplicates(subset='user_id', inplace=True)
    start_sum = df['nStarted'].sum()
    exp_sum = df['nExplored'].sum()
    comp_sum = df['nCompleted'].sum()
    return [float(comp_sum)/start_sum,float(exp_sum)/start_sum]

## Suppression

We write a general function that takes a dataframe, a list of quasi-identifiers, and a value `k`. The dataframe must be prepped with the `nStarted`, `nCompleted`, and `nExplored` fields to maintain analysis. 

In [379]:
"""
Returns a df where under less than k same quasi-identier samples are suppressed, making
the df k-anonymous.
"""
def suppressKAnon(df, qis, k=5):
    df_drop = df.drop_duplicates(subset=['user_id']+qis)
    #sum completion statistics and count unique_ids (k) for set of qis
    df_kanon = df_drop.groupby(qis).agg({'nStarted':'sum','nCompleted':'sum', 
                                    'nExplored':'sum','user_id':'nunique'}).reset_index()
    df_kanon = df_kanon.rename(columns={'user_id' : 'k'})
    df_kanon = df_kanon[df_kanon['k'] >= k] #suppresses less than k samples
    
    #print statistics
    stats = getStats(df_kanon)
    cr_anon = stats[0]
    er_anon = stats[1]
    print(str(k)+"-anon dataset completion rate: %.3f%%"%(cr_anon*100))
    print(str(k)+"-anon dataset exploration rate: %.3f%%"%(er_anon*100))
    
    #now must re-add records based on k
    df_kanon = pd.DataFrame(np.repeat(df_kanon.values,df_kanon['k'].values,
                                     axis=0), columns=df_kanon.columns)
    
    #must drop k, nStarted and nCompleted fields as these are artifacts of
    #completion analysis
    df_kanon = df_kanon.drop(columns=['k','nStarted','nCompleted'])
    
    #print number of records suppresses
    records_suppressed = len(df)-len(df_kanon)
    print(str(records_suppressed)+" records suppressed for k="+str(k))
    return df_kanon.sort_values(by=qis)

### Completion Statistics

In [373]:
clean_stats = getStats(df_clean);

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [377]:
print("Clean dataset completion rate: %.3f%%"%(clean_stats[0]*100))
print("Clean dataset exploration rate: %.3f%%"%(clean_stats[1]*100))

Clean dataset completion rate: 2.777%
Clean dataset exploration rate: 13.430%


In [380]:
df_3supp = suppressKAnon(df_clean, qis, 3)

3-anon dataset completion rate: 1.279%
3-anon dataset exploration rate: 10.638%
183150 records suppressed for k=3


In [381]:
df_4supp = suppressKAnon(df_clean, qis, 4)

4-anon dataset completion rate: 1.210%
4-anon dataset exploration rate: 10.388%
187797 records suppressed for k=4


In [382]:
df_5supp = suppressKAnon(df_clean, qis, 5)

5-anon dataset completion rate: 1.214%
5-anon dataset exploration rate: 10.275%
190473 records suppressed for k=5


In [201]:
df_clean[(df_clean['YoB']==1975)&(df_clean['gender']=='m')&(df_clean['city']=='Abu Dhabi')]

,user_id,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events,nStarted,nCompleted
104639,133606,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,0,b,1975.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0
102509,133606,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,0,b,1975.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0
88808,764663,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,0,p_oth,1975.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
47715,1334714,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,0,m,1975.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0


In [210]:
df_clean[(df_clean['YoB']==1967)&(df_clean['gender']=='f')&(df_clean['city']=='Abu Dhabi')]

,user_id,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,LoE,YoB,...,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events,nStarted,nCompleted,k
165164,9844893,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,0,b,1967.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0
158529,10749635,AE,United Arab Emirates,Asia,Abu Dhabi,AZ,Abu Dhabi,0,b,1967.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0


In [228]:
df_clean[(df_clean['YoB']==1992)&(df_clean['gender']=='m')&(df_clean['city']=='Brisbane')&
        (df_clean['LoE']=='b')]

,user_id,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events,nStarted,nCompleted
113265,1183572,AU,Australia,Oceania,Brisbane,QLD,Queensland,0,b,1992.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
76262,4710913,AU,Australia,Oceania,Brisbane,QLD,Queensland,0,b,1992.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
182657,5620916,AU,Australia,Oceania,Brisbane,QLD,Queensland,0,b,1992.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
17573,6335153,AU,Australia,Oceania,Brisbane,QLD,Queensland,0,b,1992.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1.0
104005,6335153,AU,Australia,Oceania,Brisbane,QLD,Queensland,0,b,1992.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1.0
197944,7774303,AU,Australia,Oceania,Brisbane,QLD,Queensland,4169,b,1992.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
695,9240241,AU,Australia,Oceania,Brisbane,QLD,Queensland,0,b,1992.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0
187752,9240241,AU,Australia,Oceania,Brisbane,QLD,Queensland,0,b,1992.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0
111507,9240241,AU,Australia,Oceania,Brisbane,QLD,Queensland,0,b,1992.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0
188170,14393270,AU,Australia,Oceania,Brisbane,QLD,Queensland,0,b,1992.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0


## Synthetic Records

In [387]:
def syntheticKAnon(df, qis, k=5):
    #drops duplicates to avoid double counting for the completion rate
    df_drop = df.drop_duplicates(subset=['user_id']+qis)
    #sum completion statistics and count unique_ids (k) for set of qis
    df_kanon = df_drop.groupby(qis).agg({'nStarted':'sum','nCompleted':'sum', 
                                    'nExplored':'sum','user_id':'nunique'}).reset_index()
    df_kanon = df_kanon.rename(columns={'user_id' : 'k'})
    df_add_synth = df_kanon[df_kanon['k'] < k] #df to add records to
    df_kanon = df_kanon[df_kanon['k']>=k] #df which doesnt need synthetic records
    
    #add synthetic records based on k (doesnt include original record so k+1)
    df_add_synth = pd.DataFrame(np.repeat(df_add_synth.values,
                                          k+1 - df_add_synth['k'].values, 
                                     axis=0), columns=df_add_synth.columns)
    
    df_kanon = df_kanon.append(df_add_synth, ignore_index=True) #combine datasets
    
    #print statistics
    stats = getStats(df_kanon)
    cr_anon = stats[0]
    er_anon = stats[1]
    print(str(k)+"-anon dataset completion rate: %.3f%%"%(cr_anon*100))
    print(str(k)+"-anon dataset exploration rate: %.3f%%"%(er_anon*100))
    
    #must drop k, nStarted and nCompleted fields as these are artifacts of
    #completion analysis
    df_kanon = df_kanon.drop(columns=['k','nStarted','nCompleted'])
    
    #print number of records added
    records_added = len(df_kanon)-len(df)
    print(str(records_added)+" records added for k="+str(k))
    
    return df_kanon.sort_values(by=qis)

### Completion Statistics

In [388]:
df_3synth = syntheticKAnon(df_clean, qis, 3)

3-anon dataset completion rate: 3.922%
3-anon dataset exploration rate: 15.762%
98071 records added for k=3


In [389]:
df_4synth = syntheticKAnon(df_clean, qis, 4)

4-anon dataset completion rate: 3.909%
4-anon dataset exploration rate: 15.741%
199575 records added for k=4


In [390]:
df_5synth = syntheticKAnon(df_clean, qis, 5)

5-anon dataset completion rate: 3.888%
5-anon dataset exploration rate: 15.705%
301748 records added for k=5


## Generalization, Blurring, and Suppression

In [330]:
df_clean

,user_id,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events,nStarted,nCompleted
194141,1,US,United States,Americas,Cambridge,MA,Massachusetts,02138,p,0.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
41519,8,US,United States,Americas,Cambridge,MA,Massachusetts,02139,p,1959.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
20131,11,US,United States,Americas,Monson,MA,Massachusetts,01057,NaN,1980.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
172418,30,US,United States,North America,NaN,NaN,NaN,0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
84114,64,US,United States,Americas,Stanford,CA,California,94305,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
34681,79,US,United States,Americas,Kirkland,WA,Washington,98033,NaN,1974.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
107797,117,US,United States,Americas,Minneapolis,MN,Minnesota,55414,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
163100,120,US,United States,North America,NaN,NaN,NaN,0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
8266,194,US,United States,Americas,Santa Rosa,CA,California,95401,NaN,0.0,NaN,3.0,0.0,0.0,2.0,1.0,0.0,167.0,1,1.0
14526,210,MX,Mexico,Americas,Pachuca,HID,Hidalgo,42080,b,1985.0,m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
